In [2]:
import supabase
import geopandas as gpd

In [3]:
import tomli
with open("..\\.streamlit\\secrets.toml",'rb') as f:
	config = tomli.load(f)

client = supabase.create_client(config['supabase_url'],config['supabase_key'])

In [29]:
# client.table('')
file_paths = {
	"Tranquillity Irrigation District":r"G:\Tranquillity ID-1075\GIS\Feature\TID.shp",
	"Fresno Slough Water District":r"G:\Tranquillity ID-1075\GIS\Feature\Fresno_Slough_WD.shp",
	# "Mask District":r"G:\Tranquillity ID-1075\GIS\mask_district.shp",
	"Waterways":r"G:\Tranquillity ID-1075\GIS\Feature\waterways.shp",
}

boundaries = {k:gpd.read_file(v,epsg='4326') for k,v in file_paths.items()}
# boundaries[
# 	# 'Mask District'
# 	"Waterways"
# 	].plot()

def clean_gdfs(gdf,name):
	gdf['file_name'] = name
	try:
		gdf = gdf.set_crs("EPSG:4326")
	except:
		gdf = gdf.to_crs("EPSG:4326")
	return gdf[['geometry','file_name']]

# boundaries['Tranquillity Irrigation District'].to_sql
import pandas as pd
gdf = pd.concat([clean_gdfs(v,k) for k,v in boundaries.items()]).pipe(gpd.GeoDataFrame)
gdf#.plot()
# gdf.to_wkb()

,geometry,file_name
0,"POLYGON ((-120.28745 36.66908, -120.28733 36.6...",Tranquillity Irrigation District
0,"MULTIPOLYGON (((-120.30308 36.63930, -120.3041...",Fresno Slough Water District
0,"LINESTRING (-120.28887 36.69236, -120.28819 36...",Waterways
1,"LINESTRING (-120.34681 36.75324, -120.34656 36...",Waterways
2,"LINESTRING (-120.22902 36.32530, -120.22998 36...",Waterways
3,"LINESTRING (-120.17498 36.56440, -120.17448 36...",Waterways


In [43]:
from sqlalchemy import create_engine
def connect_to_postgres(username,password,host,database):
	engine = create_engine(f"postgresql://{username}:{password}@{host}:5432/{database}")  
	return engine

def ag_water():
	engine = connect_to_postgres(	
		username='postgres',
		password=config['password'],
		host=config['host'],
		database='postgres',
	)
	return engine
# gdf.to_wkb().to_sql('TID_gis_boundaries',ag_water(),if_exists='replace')
gdf.to_wkt().to_sql('TID_gis_boundaries',ag_water(),if_exists='replace')

6

In [38]:
gdf.to_wkb()
# gdf.to_wkt()

,geometry,file_name
0,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00o\x00\x0...,Tranquillity Irrigation District
0,b'\x01\x06\x00\x00\x00\x02\x00\x00\x00\x01\x03...,Fresno Slough Water District
0,b'\x01\x02\x00\x00\x00\x82\x00\x00\x00;\xe4O\x...,Waterways
1,b'\x01\x02\x00\x00\x00J\x00\x00\x00\x81\x06s52...,Waterways
2,b'\x01\x02\x00\x00\x00\xd5\x00\x00\x00\xc0r\x1...,Waterways
3,b'\x01\x02\x00\x00\x00(\x00\x00\x00\x18\x198\x...,Waterways


In [32]:
client.table('TID_gis_boundaries').insert(gdf.to_json()).execute()

APIResponse(data=[], count=None)

In [13]:
import pandas as pd
pd.DataFrame(
client.table('TID_subsidence_points').select('*').execute().data
)

,index,point_id,point_number,northing,easting,latitude,longitude,notes,source
0,0,AG-20,107.0,2.117338e+06,6.200248e+06,36 38 13.287343 N,120 13 54.744731 W,FD_MAG_NAIL,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
1,1,AG-24,116.0,2.129655e+06,6.189336e+06,36 40 13.671861 N,120 16 10.619000 W,FD_MAG_NAIL,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
2,2,MW-19B,109.0,2.122609e+06,6.194299e+06,36 39 04.649529 N,120 15 08.578577 W,FD_MAG_NAIL,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
3,3,MW-TW3,101.0,2.104875e+06,6.200066e+06,36 36 10.033198 N,120 13 55.012587 W,FD_MAG_NAIL,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
4,4,TID A,120.0,2.126313e+06,6.197654e+06,36 39 41.698066 N,120 14 27.992791 W,None,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
5,5,TID B,102.0,2.107582e+06,6.197043e+06,36 36 36.410696 N,120 14 32.515352 W,None,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
6,6,TID C,125.0,2.131619e+06,6.193260e+06,36 40 33.596531 N,120 15 22.778805 W,None,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
7,7,TID D,105.0,2.121070e+06,6.181000e+06,36 38 47.679974 N,120 17 51.517455 W,None,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
8,8,Martin,NaN,2.099696e+06,6.220352e+06,36 35 21.325690 N,120 09 45.463844 W,None,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
9,9,Murietta,NaN,2.115865e+06,6.175004e+06,36 37 55.412397 N,120 19 04.215121 W,None,\\ppeng.com\pzdata\clients\Tranquillity ID-107...
